In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-3
eval_iters = 100
eval_interval = 500
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

In [2]:
chars = ""
with open('TGG.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)